In [1]:
import sys
import os

# Add the path to the aqi package
aqi_path = os.path.abspath('../aqi')
sys.path.append(aqi_path)

from fetch import fetch_data
from etl import validate_data, parse_data, clean_data, push_to_db

In [7]:
aqi_data = fetch_data("6814")
print(aqi_data['status'])
print(len(aqi_data['data']))

<Response [200]>
{'status': 'ok', 'data': {'aqi': 85, 'idx': 6814, 'attributions': [{'url': 'http://aqmthai.com/', 'name': 'Division of Air Quality Data, Air Quality and Noise Management Bureau, Pollution Control Department.', 'logo': 'Thailand-PCD.png'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [13.5502631, 100.2647676], 'name': 'Provincial Administrative Organization, Samut Sakhon, Thailand (องค์การบริหารส่วนจังหวัดสมุทรสาคร)', 'url': 'https://aqicn.org/city/thailand/samut-sakhon/provincial-administrative-organization', 'location': ''}, 'dominentpol': 'pm25', 'iaqi': {'dew': {'v': -5.2}, 'h': {'v': 8.4}, 'no2': {'v': 7.5}, 'o3': {'v': 2.8}, 'p': {'v': 1012.4}, 'pm10': {'v': 33}, 'pm25': {'v': 85}, 'r': {'v': 0.6}, 'so2': {'v': 2.7}, 't': {'v': 32.7}, 'w': {'v': 0.5}}, 'time': {'s': '2025-02-23 20:00:00', 'tz': '+07:00', 'v': 1740340800, 'iso': '2025-02-23T20:00:00+07:00'}, 'forecast': {'daily': {'o3': [{'avg': 9, 'day': '2025-02-21', 

In [8]:
validate_data(aqi_data)

In [9]:
print(aqi_data["data"].get("idx"))

parse_aqi_data = parse_data(aqi_data)
print(parse_aqi_data)

6814
    idx                                            address  aqi  dew    h  \
0  6814  Provincial Administrative Organization, Samut ...   85 -5.2  8.4   

    o3       p  pm10  pm25    r     t    w                 timestamp  \
0  2.8  1012.4    33    85  0.6  32.7  0.5 2025-02-23 20:00:00+07:00   

                                            forecast  
0  {"o3": [{"avg": 9, "day": "2025-02-21", "max":...  


In [10]:
clean_aqi_data = clean_data(parse_aqi_data)
print(clean_aqi_data)

    idx                                            address  aqi  dew    h  \
0  6814  Provincial Administrative Organization, Samut ...   85 -5.2  8.4   

    o3       p  pm10  pm25    r     t    w                 timestamp  \
0  2.8  1012.4    33    85  0.6  32.7  0.5 2025-02-23 20:00:00+07:00   

                                            forecast  
0  {"o3": [{"avg": 9, "day": "2025-02-21", "max":...  


In [11]:
import psycopg2

# Database connection parameters
db_params = {
    'dbname': 'air_data',
    'user': 'admin',
    'password': 'adminpassword',
    'host': 'localhost',
    'port': '54329'
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Push the cleaned AQI data to the database
push_data = push_to_db(conn, clean_aqi_data)

# Close the connection
conn.close()